<a href="https://colab.research.google.com/github/Arnab1702053/websiteforautism/blob/main/Adultwebsite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
pip install streamlit==1.13.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [25]:
cd /drive/MyDrive/adultwebsite

/drive/MyDrive/adultwebsite


In [19]:
from google.colab import files
uploaded=files.upload()

KeyboardInterrupt: ignored

In [26]:
import pandas as pd

# Not limiting the column number when displaying dataframe
pd.set_option("display.max_columns", None)

In [27]:
from sklearn import preprocessing

df = pd.read_csv("Autism_Data.arff", sep = r',', skipinitialspace = True)

df = df.fillna(0)

df = df.drop(["used_app_before"], axis = 1)

df = df.rename(columns = {"austim": "autism", "contry_of_res": "country"})
df = df.replace("yes", 1)
df = df.replace("no", 0)
df = df.replace("f", 1)
df = df.replace("m", 0)
df = df.replace("YES", '   ASD Trait')
df = df.replace("NO", '   Not ASD Trait')
df = df.replace("?", 0)

In [28]:
# Typecasting
df["ethnicity"] = df["ethnicity"].astype(str)
df["country"] = df["country"].astype(str)
df["age_desc"] = df["age_desc"].astype(str)
df["relation"] = df["relation"].astype(str)

# Initializing Encoder
number = preprocessing.LabelEncoder()

# Encoding
df["ethnicity"] = number.fit_transform(df["ethnicity"])
df["country"] = number.fit_transform(df["country"])
df["age_desc"] = number.fit_transform(df["age_desc"])
df["relation"] = number.fit_transform(df["relation"])

df.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,autism,country,result,age_desc,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,26,1,10,0,0,10,6,0,5,Not ASD Trait
1,1,1,0,1,0,0,0,1,0,1,24,0,6,0,1,25,5,0,5,Not ASD Trait
2,1,1,0,1,1,0,1,1,1,1,27,0,6,1,1,61,8,0,3,ASD Trait
3,1,1,0,1,0,0,1,1,0,1,35,1,10,0,1,10,6,0,5,Not ASD Trait
4,1,0,0,0,0,0,0,1,0,0,40,1,2,0,0,32,2,0,1,Not ASD Trait


In [29]:
df.shape

(704, 20)

In [31]:
X = df[['A1_Score','A2_Score','A3_Score','A4_Score','A5_Score','A6_Score','A7_Score','A8_Score','A9_Score','A10_Score','age','gender']].values
y = df["Class/ASD"].values


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.35)

In [32]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [33]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver = "liblinear")
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

1.0


In [34]:
clf= LogisticRegression()
clf.fit(X_test,y_test)

LogisticRegression()

In [35]:
import joblib
joblib.dump(clf,"clf.pkl")

['clf.pkl']

In [36]:
%%writefile app.py
import pandas as pd

import streamlit as st
import pandas as pd
import joblib

# Title
st.header("Autism Detection App For Adult")


A1_Score=st.selectbox('Q1. I often notice small sounds when others do not ',['yes','no'])

A2_Score=st.selectbox('Q2. I usually concentrate more on the whole picture, rather than the small details ',['yes','no'])

A3_Score=st.selectbox('Q3. I find it easy to do more than one thing at once',['yes','no'])

A4_Score=st.selectbox('Q4. If there is an interruption, I can switch back to what I was doing very quickly',['yes','no'])

A5_Score=st.selectbox('Q5. I find it easy to ‘read between the lines’ when someone is talking to me ',['yes','no'])

A6_Score=st.selectbox('Q6. I know how to tell if someone listening to me is getting bored ',['yes','no'])

A7_Score=st.selectbox('Q7. When I’m reading a story I find it difficult to work out the characters’ intentions',['yes','no'])

A8_Score=st.selectbox('Q8. I like to collect information about categories of things (e.g. types of car, types of bird, types of train, types of plant etc)',['yes','no'])

A9_Score=st.selectbox('Q9.I find it easy to work out what someone is thinking or feeling just by looking at their face ',['yes','no'])

A10_Score=st.selectbox('Q10.I find it difficult to work out people’s intentions',['yes','no'])

# Input bar 1
age = st.number_input("Enter Age", min_value=18, max_value=100)

# Input bar 2
#weight = st.number_input("Enter Weight")

# Dropdown input
#eyes = st.selectbox("Select Eye Colour", ("Blue", "Brown"))
gender= st.selectbox("Select Sex", ("Male", "Female"))

# If button is pressed
#if st.button("Submit")
if st.button('submit'):
  clf=joblib.load('clf.pkl')
  X_test= pd.DataFrame([[A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender]], columns=['A1_Score','A2_Score','A3_Score','A4_Score','A5_Score','A6_Score','A7_Score','A8_Score','A9_Score','A10_Score','age','gender'])
  X_test=X_test.replace(['yes','no'],[1,0])
  X_test=X_test.replace(['Male','Female'],[1,0])
  prediction=clf.predict(X_test)[0]
  st.text(f"this is{prediction}")  

    
    

Writing app.py


In [37]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
from pyngrok import ngrok

In [39]:
ngrok.set_auth_token('2KHbIDb5aXAIAeyu5vBsPKaa7Al_6kJG3JFwwFfhEdSTBgkks')

In [40]:
ngrok.kill()

In [41]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://86df-35-221-26-203.ngrok.io" -> "http://localhost:80"


In [43]:
from pyngrok import ngrok
tunnels=ngrok.get_tunnels()
print(tunnels)

[<NgrokTunnel: "http://86df-35-221-26-203.ngrok.io" -> "http://localhost:80">, <NgrokTunnel: "https://86df-35-221-26-203.ngrok.io" -> "http://localhost:80">]
